# 1 Import libraries

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
import datetime
import pytz
import IPython

In [2]:
from pandas import DataFrame
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle

In [3]:
print('TF version:', tf.__version__)
print('GPU devices:', tf.config.list_physical_devices('GPU'))

TF version: 2.8.2
GPU devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# 2 Data load and preprocessing

In [4]:
from tensorflow.keras.utils import to_categorical

def prepare_data():
  # columns used 
  columns = ['sepal length in cm', 'sepal width in cm', 'petal length in cm', 'petal width in cm', 'class']

	# generate 2d classification dataset
  income = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data",
                   header=None,
                   names=columns)

  # shuffle arrays because 'class' column is organized in order
  df = shuffle(income.values)
  X = df[:,0:4]
  Y = df[:,4]
	
  # encode class values as integers
  encoder = LabelEncoder()
  encoder.fit(Y)
  encoded_Y = encoder.transform(Y)
  # convert integers to dummy variables (i.e. one hot encoded)
  dummy_y = np_utils.to_categorical(encoded_Y)
	
  # split into train and test
  n_train = 105
  train_x, test_x = X[:n_train, :], X[n_train:, :]
  train_y, test_y = dummy_y[:n_train], dummy_y[n_train:]

  train_x = np.asarray(train_x).astype(np.float32)
  train_y = np.asarray(train_y).astype(np.float32)
  test_x = np.asarray(test_x).astype(np.float32)
  test_y = np.asarray(test_y).astype(np.float32)

  return train_x, train_y, test_x, test_y

In [5]:
# Loading the data 
train_x, train_y, test_x, test_y = prepare_data()

In [6]:
class MyCustomCallback(tf.keras.callbacks.Callback):

  def on_train_begin(self, batch, logs=None):
    self.begins = time.time()
    print('Training: begins at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))

  def on_train_end(self, logs=None):
    print('Training: ends at {}'.format(datetime.datetime.now(pytz.timezone('America/Fortaleza')).strftime("%a, %d %b %Y %H:%M:%S")))
    print('Duration: {:.2f} seconds'.format(time.time() - self.begins))    

# 3 Hyperparameter Tuning 

In [7]:
%%capture
!pip install wandb==0.10.17

In [8]:
!wandb login --relogin

wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 3.1 Monitoring a neural network

In [9]:
import wandb
from wandb.keras import WandbCallback
from tensorflow.keras.callbacks import EarlyStopping

# Default values for hyperparameters
defaults = dict(layer_1 = 8,
                layer_2 = 8,
                learn_rate = 0.01,
                batch_size = 32,
                epoch = 500)

wandb.init(project="projetoii", config= defaults, name="projetoii_run_01")
config = wandb.config

wandb: Currently logged in as: danibmarques (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


In [19]:
# Instantiate a simple classification model
model = tf.keras.Sequential([
  tf.keras.layers.Dense(config.layer_1, activation=tf.nn.relu, dtype='float32'),
  tf.keras.layers.Dense(config.layer_2, activation=tf.nn.relu, dtype='float32'),
  tf.keras.layers.Dense(3, activation=tf.nn.softmax, dtype='float32')
])

# Instantiate a logistic loss function that expects integer targets.
loss = tf.keras.losses.CategoricalCrossentropy()

# Instantiate an accuracy metric.
accuracy = tf.keras.metrics.CategoricalCrossentropy()

# Instantiate an optimizer.
optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate)

# configure the optimizer, loss, and metrics to monitor.
model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])

In [ ]:
%%wandb
# Add WandbCallback() to the fit function
model.fit(x=train_x,
          y=train_y,
          batch_size=config.batch_size,
          epochs=config.epoch,
          validation_data=(test_x,test_y),
          callbacks=[WandbCallback(log_weights=True)],
          verbose='auto')


## 3.2 Sweeps

In [21]:
 # The sweep calls this function with each set of hyperparameters
def train():
    # Default values for hyper-parameters we're going to sweep over
    defaults = dict(layer_1 = 8,
                layer_2 = 8,
                learn_rate = 0.01,
                batch_size = 32,
                epoch = 500)
    
    # Initialize a new wandb run
    wandb.init(project="projetoii", config= defaults)

    # Config is a variable that holds and saves hyperparameters and inputs
    config = wandb.config
    
    # Instantiate a simple classification model
    model = tf.keras.Sequential([
                                 tf.keras.layers.Dense(config.layer_1, activation=tf.nn.relu, dtype='float32'),
                                 tf.keras.layers.Dense(config.layer_2, activation=tf.nn.relu, dtype='float32'),
                                 tf.keras.layers.Dense(3, activation=tf.nn.softmax, dtype='float32')
                                 ])

    # Instantiate a logistic loss function that expects integer targets.
    loss = tf.keras.losses.CategoricalCrossentropy()

    # Instantiate an accuracy metric.
    accuracy = tf.keras.metrics.CategoricalCrossentropy()

    # Instantiate an optimizer.
    optimizer = tf.keras.optimizers.SGD(learning_rate=config.learn_rate)

    # configure the optimizer, loss, and metrics to monitor.
    model.compile(optimizer=optimizer, loss=loss, metrics=[accuracy])  

    model.fit(train_x, train_y, batch_size=config.batch_size,
              epochs=config.epoch,
              validation_data=(test_x, test_y),
              callbacks=[WandbCallback(),
                          EarlyStopping(patience=100)]
              )   

In [22]:
# Configure the sweep – specify the parameters to search through, the search strategy, the optimization metric et all.
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'categorical_accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'layer_1': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'layer_2': {
            'max': 32,
            'min': 8,
            'distribution': 'int_uniform',
        },
        'learn_rate': {
            'min': -4,
            'max': -2,
            'distribution': 'log_uniform',  
        },
        'epoch': {
            'values': [300,400,600]
        },
        'batch_size': {
            'values': [32,64]
        }
    }
}

In [23]:
# Initialize a new sweep
# Arguments:
#     – sweep_config: the sweep config dictionary defined above
#     – entity: Set the username for the sweep
#     – project: Set the project name for the sweep
sweep_id = wandb.sweep(sweep_config, entity="danibmarques", project="projetoii")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: ikijkj2c
Sweep URL: https://wandb.ai/danibmarques/projetoii/sweeps/ikijkj2c


In [24]:
# Initialize a new sweep
# Arguments:
#     – sweep_id: the sweep_id to run - this was returned above by wandb.sweep()
#     – function: function that defines your model architecture and trains it
wandb.agent(sweep_id = sweep_id, function=train,count=20)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: w9q78ej2 with config:
wandb: 	batch_size: 32
wandb: 	epoch: 300
wandb: 	layer_1: 30
wandb: 	layer_2: 31
wandb: 	learn_rate: 0.11610828446658154
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/300
4/4 [==============================] - 1s 64ms/step - loss: 1.7033 - categorical_crossentropy: 1.7033 - val_loss: 0.9256 - val_categorical_crossentropy: 0.9256
Epoch 2/300
4/4 [==============================] - 0s 14ms/step - loss: 0.7875 - categorical_crossentropy: 0.7875 - val_loss: 0.6514 - val_categorical_crossentropy: 0.6514
Epoch 3/300
4/4 [==============================] - 0s 17ms/step - loss: 0.5503 - categorical_crossentropy: 0.5503 - val_loss: 0.5358 - val_categorical_crossentropy: 0.5358
Epoch 4/300
4/4 [==============================] - 0s 10ms/step - loss: 0.4608 - categorical_crossentropy: 0.4608 - val_loss: 0.5923 - val_categorical_crossentropy: 0.5923
Epoch 5/300
4/4 [==============================] - 0s 12ms/step - loss: 0.6729 - categorical_crossentropy: 0.6729 - val_loss: 0.5419 - val_categorical_crossentropy: 0.5419
Epoch 6/300
4/4 [==============================] - 0s 14ms/step - loss: 0.4307 - categorical_crossentropy: 0.4307 - val_loss: 0.4248 - val_c

epoch,299
loss,0.07006
categorical_crossentropy,0.07006
val_loss,0.24696
val_categorical_crossentropy,0.24696
_step,299
_runtime,20
_timestamp,1658539306
best_val_loss,0.08017
best_epoch,298


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,▅▄█▃▂▄▃▆▂▅▁▅▃▂▂▂▂▁▂▁▁▁▂▁▁▁▁▅▁▁▁▂▁▁▁▂▁▁▁▁
categorical_crossentropy,▅▄█▃▂▄▃▆▂▅▁▅▃▂▂▂▂▁▂▁▁▁▂▁▁▁▁▅▁▁▁▂▁▁▁▂▁▁▁▁
val_loss,▃▂▃▃▄▇▂▂█▂▁▂▁▂▂▅▁▁▂▁▁▁▁▃▂▁▁▁▁▁▁▄▂▁▃▂▂▃▁▂
val_categorical_crossentropy,▃▂▃▃▄▇▂▂█▂▁▂▁▂▂▅▁▁▂▁▁▁▁▃▂▁▁▁▁▁▁▄▂▁▃▂▂▃▁▂
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇████


wandb: Agent Starting Run: xciv5ljg with config:
wandb: 	batch_size: 64
wandb: 	epoch: 400
wandb: 	layer_1: 30
wandb: 	layer_2: 10
wandb: 	learn_rate: 0.05806231378738195
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


...

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂
categorical_crossentropy,█▇▅▄▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁▁▂
val_loss,█▇▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_categorical_crossentropy,█▇▅▄▄▃▃▃▂▂▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 9elptaak with config:
wandb: 	batch_size: 64
wandb: 	epoch: 300
wandb: 	layer_1: 15
wandb: 	layer_2: 30
wandb: 	learn_rate: 0.06340506910537098
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: wandb version 0.12.21 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/300
2/2 [==============================] - 1s 201ms/step - loss: 2.0395 - categorical_crossentropy: 2.0395 - val_loss: 1.1619 - val_categorical_crossentropy: 1.1619
Epoch 2/300
2/2 [==============================] - 0s 46ms/step - loss: 1.0438 - categorical_crossentropy: 1.0438 - val_loss: 0.9020 - val_categorical_crossentropy: 0.9020
Epoch 3/300
2/2 [==============================] - 0s 44ms/step - loss: 0.8752 - categorical_crossentropy: 0.8752 - val_loss: 0.8389 - val_categorical_crossentropy: 0.8389
Epoch 4/300
2/2 [==============================] - 0s 47ms/step - loss: 0.8174 - categorical_crossentropy: 0.8174 - val_loss: 0.8275 - val_categorical_crossentropy: 0.8275
Epoch 5/300
2/2 [==============================] - 0s 48ms/step - loss: 0.7679 - categorical_crossentropy: 0.7679 - val_loss: 0.7560 - val_categorical_crossentropy: 0.7560
Epoch 6/300
2/2 [==============================] - 0s 52ms/step - loss: 0.7207 - categorical_crossentropy: 0.7207 - val_loss: 0.7270 - val_

epoch,299
loss,0.16558
categorical_crossentropy,0.16558
val_loss,0.13161
val_categorical_crossentropy,0.13161
_step,299
_runtime,21
_timestamp,1658540009
best_val_loss,0.08064
best_epoch,266


epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▇▅▄▄▄▃▃▃▄▃▃▂▂▆▂▃▁▂▁▁▁▂▁▁▁▁▃▁▂▁▂▁▁▁▁▁▁▁▂
categorical_crossentropy,█▇▅▄▄▄▃▃▃▄▃▃▂▂▆▂▃▁▂▁▁▁▂▁▁▁▁▃▁▂▁▂▁▁▁▁▁▁▁▂
val_loss,█▆▅▄▄▅▅▃▃▃▄▄▂▂▆▂▄▁▂▂▄▁▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
val_categorical_crossentropy,█▆▅▄▄▅▅▃▃▃▄▄▂▂▆▂▄▁▂▂▄▁▂▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
_runtime,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████
_timestamp,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▆▇▇▇▇████


### 3.2.1 Restore a model

Restore a file, such as a model checkpoint, into your local run folder to access in your script.

See [the restore docs](https://docs.wandb.com/library/restore) for more details.

In [27]:
%%capture
!pip install wandb==0.10.17

In [28]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [29]:
 import wandb
 wandb.__version__

'0.10.17'

In [30]:
!wandb login

wandb: Currently logged in as: danibmarques (use `wandb login --relogin` to force relogin)


In [31]:
import wandb
# restore the raw model file "model-best.h5" from a specific run by user "danibmarques"
# in project "projetoii" from run "ikijkj2c"
best_model = wandb.restore('model-best.h5', run_path="danibmarques/projetoii/hpfm0emv")

In [32]:
# restore the model for tf.keras
model = tf.keras.models.load_model(best_model.name)

In [33]:
# execute the loss and accuracy using the test dataset
loss_, acc_ = model.evaluate(x=test_x,y=test_y, batch_size=64)
print('Test loss: %.3f - acc: %.3f' % (loss_, acc_))

1/1 [==============================] - 0s 126ms/step - loss: 0.0911 - categorical_crossentropy: 0.0911
Test loss: 0.091 - acc: 0.091


# 4 References

1. https://github.com/wandb/awesome-dl-projects
2. https://docs.wandb.ai/app/features/panels/parameter-importance
3. https://wandb.ai/wandb/DistHyperOpt/reports/Modern-Scalable-Hyperparameter-Tuning-Methods--VmlldzoyMTQxODM